In [ ]:
# MP 2.2. Using linear system of equation to solve physical problems

import numpy as np

u3 = 1
u5 = 1
u6 = -1
F6 = -10
u1, u2, u4 = 0, 0, 0
F1, F2, F3, F4, F5 = 0, 0, 0, 0, 0

x = np.array([u1,u2,u3,u4,u5,u6])
F = np.array([F1,F2,F3,F4,F5,F6])


In [ ]:
# MP 2.4. Strategy to solve KU=P

import numpy as np

def reorder_rows_columns(K,equation_numbers):
    # construct the matrix Khat
    Khat_row = np.zeros_like(K)
    Khat = np.zeros_like(K)
    for row_index, equation_number in enumerate(equation_numbers):
        Khat_row[row_index] = K[equation_number-1]
    for col_index, equation_number in enumerate(equation_numbers):
        Khat[:,col_index] = Khat_row[:,equation_number-1]
    return Khat

In [ ]:
# MP 2.5. Strategy to solve KU=P (continue)

# Input
nk = 3
"""
nk: number of known prescribed displacements
nf: number of unknown displacements (the ones we are trying to solve for)
xp: prescribed (known) displacements with shape (nk,)
xf: unknown displacements with shape (nf,)
Fp: reaction (unknown) forces with shape (nk,)
Ff: applied (known) forces with shape (nf,)
Kff: block of stiffness matrix with shape (nf,nf)
Kfp: block of stiffness matrix with shape (nf,nk)
Kpf: block of stiffness matrix with shape (nk,nf)
Kpp: block of stiffness matrix with shape (nk,nk)
"""

import numpy as np

def partition_stiffness_matrix(K,equation_numbers,nk):
    # construct the smaller matrices
    Khat = reorder_rows_columns(K, equation_numbers)
    Kpp = Khat[:nk,:nk]
    Kpf = Khat[:nk,nk:]
    Kfp = Khat[nk:,:nk]
    Kff = Khat[nk:,nk:]
    return  Kpp,Kpf,Kfp,Kff

In [ ]:
# MP 2.6. Solving KU=P

import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

def reorder_rows_columns(K,equation_numbers):
    # construct the matrix Khat
    Khat_row = np.zeros_like(K)
    Khat = np.zeros_like(K)
    for row_index, equation_number in enumerate(equation_numbers):
        Khat_row[row_index] = K[equation_number-1]
    for col_index, equation_number in enumerate(equation_numbers):
        Khat[:,col_index] = Khat_row[:,equation_number-1]
    return Khat

def partition_stiffness_matrix(K,equation_numbers,nk):
    # construct the smaller matrices
    Khat = reorder_rows_columns(K, equation_numbers)
    Kpp = Khat[:nk,:nk]
    Kpf = Khat[:nk,nk:]
    Kfp = Khat[nk:,:nk]
    Kff = Khat[nk:,nk:]
    return  Kpp,Kpf,Kfp,Kff

def fea_solve(Kpp,Kpf,Kfp,Kff,xp,Ff):
    # do stuff here
    xf = la.solve(Kff, Ff-np.dot(Kfp, xp))
    Fp = np.dot(Kpp,xp)+np.dot(Kpf,xf)
    return xf,Fp

nk = 3
Kpp,Kpf,Kfp,Kff = partition_stiffness_matrix(K,equation_numbers,nk)
xp = np.array([0,0,0],dtype='float64')
Ff = np.array([0,0,-10],dtype='float64')

xf,Fp = fea_solve(Kpp,Kpf,Kfp,Kff,xp,Ff)

image_xf = plot_truss(xf)

In [ ]:
# MP 2.7. LU and Triangular Solve

import numpy as np


def my_lu(A):
    # The upper triangular matrix U is saved in the upper part of the matrix M (including the diagonal)
    # The lower triangular matrix L is saved in the lower part of the matrix M (not including the diagonal)
    # Do NOT use `scipy.linalg.lu`!
    # You should not use pivoting
    
    # LU Factorization
    M = A.copy()
    for i in range(A.shape[0]-1):
        for j in range(i+1,A.shape[0]):
            M[j, i] = M[j, i] / M[i, i]
            for k in range(i+1,A.shape[0]):
                M[j,k] -= M[i,k]*M[j,i]
    return M


def my_triangular_solve(M, b):
    # A = LU (L and U are stored in M)
    # A x = b (given A and b, find x)
    # M is a 2D numpy array
    # The upper triangular matrix U is stored in the upper part of the matrix M (including the diagonal)
    # The lower triangular matrix L is stored in the lower part of the matrix M (not including the diagonal)
    # b is a 1D numpy array
    # x is a 1D numpy array
    # Do not use `scipy.linalg.solve_triangular`

    # LUx = b
    # 1. solve Ly = b, where Ux = y
    # 2. solve Ux = y
    
    x = np.zeros(b.shape[0])
    y = np.zeros(b.shape[0])
    z = np.zeros((b.shape[0] ,b.shape[0]))
    for i in range(b.shape[0]):
        z[i, i] = 1
        for j in range(i):
            z[i,j] = M[i,j]
    for i in range(b.shape[0]):
        t = b[i]
        for j in range(i):
            t -= y[j] * z[i,j]
        y[i] = t/z[i,i]
    for i in range(b.shape[0]-1, -1, -1):
        t = y[i]
        for j in range(i, b.shape[0]):
            t -= x[j] * M[i,j]
        x[i] = t/M[i,i]
    return x


def fea_solve(Kpp, Kpf, Kfp, Kff, xp, Ff):
    # Use my_lu and my_triangular_solve

    xf = my_triangular_solve(my_lu(Kff), Ff - Kfp @ xp)
    Fp = Kpp @ xp + Kpf @ xf
    return xf, Fp